# BM25 Algorithm Explanation and Implementation

### 1. Introduction to BM25

BM25 (Best Matching 25) is a ranking function used in information retrieval to estimate the relevance of documents to a given search query. It's an improvement over the basic TF-IDF weighting scheme.

The main idea behind BM25 is to rank documents based on the query terms appearing in each document, considering:
1. Term frequency (TF)
2. Inverse document frequency (IDF)
3. Document length normalization

The BM25 formula is:

$$
\text{score}(D, Q) = \sum \left( \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k1 + 1)}{f(q_i, D) + k1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})} \right)
$$

Where:
- D is a document
- Q is a query containing terms q_i
- f(q_i, D) is the frequency of q_i in D
- |D| is the length of document D
- avgdl is the average document length in the corpus
- k1 and b are free parameters (usually k1 ∈ [1.2, 2.0] and b = 0.75)

### 2. Implementing BM25

In [ ]:
import math

In [ ]:
class BM25:
    def __init__(self, corpus: list[list[str]], k1: float = 1.2, b: float = 0.75):
        self.corpus = corpus
        self.k1 = k1
        self.b = b
        self.avgdl = sum(len(doc) for doc in corpus) / len(corpus)
        self.doc_freqs = self._calculate_doc_freqs()
        self.idf = self._calculate_idf()
        self.doc_lengths = [len(doc) for doc in corpus]

    def _calculate_doc_freqs(self) -> dict[str, int]:
        doc_freqs = {}
        for doc in self.corpus:
            for term in set(doc):
                doc_freqs[term] = doc_freqs.get(term, 0) + 1
        return doc_freqs

    def _calculate_idf(self) -> dict[str, float]:
        idf = {}
        num_docs = len(self.corpus)
        for term, freq in self.doc_freqs.items():
            idf[term] = math.log((num_docs - freq + 0.5) / (freq + 0.5) + 1)
        return idf

    def _score_document(self, query: list[str], doc_index: int) -> float:
        score = 0
        doc = self.corpus[doc_index]
        doc_len = self.doc_lengths[doc_index]

        for term in query:
            if term not in doc:
                continue
            tf = doc.count(term)
            numerator = self.idf[term] * tf * (self.k1 + 1)
            denominator = tf + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)
            score += numerator / denominator

        return score

    def rank_documents(self, query: list[str]) -> list[int]:
        scores = [self._score_document(query, i) for i in range(len(self.corpus))]
        return sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

### 3. Example Usage

In [ ]:
# Sample corpus
corpus = [
    ["hello", "world", "hello", "there"],
    ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"],
    ["information", "retrieval", "is", "the", "science", "of", "searching", "for", "information"],
    ["machine", "learning", "is", "a", "subset", "of", "artificial", "intelligence"],
]

# Initialize BM25
bm25 = BM25(corpus)

# Example query
query = ["information", "retrieval"]

# Rank documents
ranked_docs = bm25.rank_documents(query)

print("Ranked document indices:", ranked_docs)

Ranked document indices: [2, 0, 1, 3]


### 4. Explanation of the Implementation

Let's break down the implementation:

1. The BM25 class is initialized with a corpus, and optional parameters k1 and b.
2. In the constructor, we calculate:
   - Average document length (avgdl)
   - Document frequencies (how many documents contain each term)
   - Inverse Document Frequency (IDF) for each term
   - Document lengths
3. The _score_document method calculates the BM25 score for a single document given a query.
4. The rank_documents method scores all documents for a given query and returns their indices sorted by relevance.

This implementation demonstrates the key components of BM25:
- Term frequency (tf) is calculated directly in _score_document
- IDF is pre-calculated in _calculate_idf
- Document length normalization is applied in _score_document

Students can experiment with different values of k1 and b to see how they affect the rankings.

### 5. (Optional) Exercises for Students

1. Implement a function to calculate precision@k for the BM25 rankings.
2. Modify the BM25 class to handle stop words removal and stemming.
3. Compare BM25 rankings with simple TF-IDF rankings. What differences do you observe?
4. Implement a variation of BM25, such as BM25+ or BM25L, and compare the results.
5. Use a real-world dataset (e.g., news articles) to test the BM25 implementation and analyze the results.

#### BM25+ Equation

$$
\text{score}(D, Q) = \sum_{i=1}^n \text{IDF}(q_i) \cdot \left(\frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})} + \delta\right)
$$

#### BM25L Equation

The BM25L scoring function for a term $t$ in a document $d$ is:

$$
\text{score}(d,t) = \text{IDF}(t) \cdot \frac{f'(t,d) \cdot (k_1 + 1)}{f'(t,d) + k_1}
$$

Where:

$$
f'(t,d) = \frac{\text{tf}'(t,d)}{1 - b + b \cdot \text{dl}'(d)}
$$

$$
\text{dl}'(d) = \frac{\text{dl}(d)}{\text{avdl}}
$$

$$
\text{tf}'(t,d) = \text{tf}(t,d) \cdot \log_2\left(1 + c \cdot \frac{\text{avdl}}{\text{dl}(d)}\right)
$$

And:

- $\text{IDF}(t)$ is the inverse document frequency of term $t$
- $\text{tf}(t,d)$ is the term frequency of $t$ in document $d$
- $\text{dl}(d)$ is the length of document $d$
- $\text{avdl}$ is the average document length in the collection
- $k_1$ and $b$ are the usual BM25 parameters
- $c$ is a new parameter (typically set to 1)

# Comparison of BM25, BM25L, and BM25+

## Formulas

### BM25
$$
\text{score}(D, Q) = \sum_{i=1}^n \text{IDF}(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}
$$

### BM25L
$$
\text{score}(D, Q) = \sum_{i=1}^n \text{IDF}(q_i) \cdot \frac{f'(q_i, D) \cdot (k_1 + 1)}{f'(q_i, D) + k_1}
$$
Where:
$$
f'(q_i, D) = \frac{\text{tf}'(q_i, D)}{1 - b + b \cdot \frac{|D|}{\text{avgdl}}}
$$
$$
\text{tf}'(q_i, D) = \text{tf}(q_i, D) \cdot \log_2\left(1 + c \cdot \frac{\text{avgdl}}{|D|}\right)
$$

### BM25+
$$
\text{score}(D, Q) = \sum_{i=1}^n \text{IDF}(q_i) \cdot \left(\frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})} + \delta\right)
$$

## Key Differences

1. **Document Length Normalization**
   - BM25: Uses $\frac{|D|}{\text{avgdl}}$ in the denominator.
   - BM25L: Incorporates document length in both $f'(q_i, D)$ and $\text{tf}'(q_i, D)$.
   - BM25+: Same as BM25, but adds $\delta$ term.

2. **Term Frequency Handling**
   - BM25: Uses raw term frequency $f(q_i, D)$.
   - BM25L: Uses modified term frequency $\text{tf}'(q_i, D)$ with logarithmic scaling.
   - BM25+: Uses raw term frequency $f(q_i, D)$, like BM25.

3. **Additional Parameters**
   - BM25: Uses $k_1$ and $b$.
   - BM25L: Introduces new parameter $c$ (typically set to 1).
   - BM25+: Introduces new parameter $\delta$ (small constant, often around 1).

## Strengths and Weaknesses

### BM25
- **Strengths**: Well-established, widely used, good general performance.
- **Weaknesses**: Can undervalue long documents, especially for terms not present.

### BM25L
- **Strengths**: Better handling of very long documents, addresses some BM25 limitations.
- **Weaknesses**: More complex, additional parameter to tune.

### BM25+
- **Strengths**: Mitigates long document undervaluation, simple modification to BM25.
- **Weaknesses**: May overcompensate for some queries, $\delta$ needs tuning.

## Use Cases

- **BM25**: General purpose, good starting point for most applications.
- **BM25L**: Collections with wide range of document lengths, especially with very long documents.
- **BM25+**: When long documents are important but BM25L's complexity is undesirable.

## Implementation Considerations

1. **Complexity**: BM25 < BM25+ < BM25L
2. **Parameter Tuning**:
   - BM25: $k_1$ and $b$
   - BM25L: $k_1$, $b$, and $c$
   - BM25+: $k_1$, $b$, and $\delta$
3. **Computational Cost**: BM25L may be slightly more expensive due to additional logarithm calculation.

## Conclusion

Each variant has its strengths and is suited to different scenarios. The choice between them depends on the specific characteristics of the document collection and the retrieval tasks at hand. Experimentation and evaluation on the target dataset are recommended to determine the best approach for a given application.